In [141]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
# from oauth2client.tools import argparser

## 1. youtube api 기본 세팅

In [ ]:
DEVEOPER_KEY="youtube api키"

In [143]:
YOUTUBE_API_SERVICE_NAME='youtube'
YOUTUBE_API_VERSION='v3'

In [144]:
youtube=build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVEOPER_KEY)

## 2. 시청 기록 data set

In [145]:
import pandas as pd
watched=pd.read_json(r"C:\Users\USER\Desktop\랩실 자료\[프로젝트] 킬링 타임 방지를 위한 유튜브 알고리즘 시각화 및 조정\코드 파일\시청 기록 파일\[정흠]시청 기록.json", encoding='utf-8')

In [146]:
YouTube_video = watched[watched['header'] == 'YouTube']

# 유튜브 뮤직은 제외
# YouTube_music=watched[watched['header'] == 'YouTube Music']

# 인덱스 초기화 (유튜브 뮤직이 빠져서 기존 인덱스 순서가 안맞음)
YouTube_video.reset_index(drop=True, inplace=True)

In [147]:
print(list(YouTube_video.columns))
print(len(YouTube_video))

['header', 'title', 'titleUrl', 'subtitles', 'time', 'products', 'activityControls']
32848


In [148]:
YouTube_video=YouTube_video[['header', 'title', 'titleUrl', 'subtitles', 'time', 'products', 'activityControls']]

In [149]:
YouTube_video.isnull().sum()

header                 0
title                  0
titleUrl             112
subtitles           1011
time                   0
products               0
activityControls       0
dtype: int64

titleUrl이 없는 행은 삭제

In [150]:
YouTube_video = YouTube_video.dropna(subset=['titleUrl'])

In [151]:
len(YouTube_video)

32736

In [152]:
import pandas as pd
# 'title' 열의 데이터 타입을 문자열로 변환
YouTube_video['title'] = YouTube_video['title'].astype(str)

In [153]:
YouTube_video=YouTube_video[YouTube_video['title']!="설문조사 질문에 답변함"]

In [154]:
len(YouTube_video)

32736

## 3. title Url에서 영상 아이디 추출

In [155]:
YouTube_video['video_id']=YouTube_video['titleUrl'].str.replace('https://www.youtube.com/watch?v=', '', regex=False)

In [156]:
len(YouTube_video)

32736

In [157]:
# YouTube_video['title'].to_csv("원본 시청 기록.csv", index=False, encoding='utf-8-sig')

## 4. 영상 정보 추출

- dislike는 api로 추출이 안됨

In [158]:
import import_ipynb
from video_info_function import *

In [159]:
import os
import pandas as pd
import time

# 이전에 저장된 데이터 불러오기
prev_df_path = r"C:\Users\USER\Desktop\랩실 자료\[프로젝트] 킬링 타임 방지를 위한 유튜브 알고리즘 시각화 및 조정\코드 파일\시청 기록 파일\[정흠]시청 기록[new].csv"
if os.path.exists(prev_df_path):
    prev_df = pd.read_csv(prev_df_path)
    start_index = len(prev_df)  # 이전 데이터의 길이를 시작 인덱스로 설정
else:
    prev_df = pd.DataFrame()
    start_index = 0

total_df = prev_df
k = 10

while True:
    detail_df = video_info(youtube, YouTube_video, start_index, start_index + k)
    
    if detail_df.empty:
        break

    total_df = pd.concat([total_df, detail_df], ignore_index=True)
    
    start_index += k

In [160]:
total_df.to_csv("[정흠]시청 기록[new].csv", index=False, encoding='utf-8-sig')

In [161]:
print("크롤링한 행개수:",len(total_df))

크롤링한 행개수: 32736


In [162]:
total_df

,title,id,date,category_id,view,like,comment,hour,min,sec,...,description,thumbnail,channel_title,tags,dimension,definition,caption,licensed_content,topic_ids,relevant_topic_ids
0,서울대 학생을 본 뉴욕대 학생 미국인 반응ㅋㅋ#shorts,jioVwVM4aG4,2023-09-27,24,236733,1889,137,0,0,25,...,#shorts #미국인 #외국인반응 #서울대 #고려대 #유학생 #서울대 반응 #뉴욕...,https://i.ytimg.com/vi/jioVwVM4aG4/default.jpg,channel CKOONY,"외국인, 외국인반응, 한국, 외국인 채널, 외국인 반응 채널, 외국인채널, 외국인 ...",2d,hd,False,True,NaN,NaN
1,타노스 과거로 가서 죽이기,XdA9rL6uvEU,2023-09-23,1,581618,19585,82,0,0,28,...,"타노스가 아기였던 시절로 돌아가 암살하면 안될까요? 퍼니셔, 코스믹 고스트 라이더인...",https://i.ytimg.com/vi/XdA9rL6uvEU/default.jpg,우라늄맨,"마블, 타노스, 코스믹고스트라이더, 코고라, 고라, 고스트라이더, 퍼니셔, 타노스죽...",2d,hd,False,False,NaN,NaN
2,"아니, 수비 하랬더니 왜 얼굴을...",vFEzDza_vfc,2023-03-27,17,183481,1558,73,0,0,57,...,NaN,https://i.ytimg.com/vi/vFEzDza_vfc/default.jpg,'B' Story,"nba, basketball, all star, 올스타, 농구, 농구 수다, 경기 ...",2d,hd,False,True,NaN,NaN
3,One Call Away는 원래 찰리푸스의 곡이 아니었다? #찰리푸스,7t5JDCzWYOY,2023-09-08,22,132684,3921,12,0,0,51,...,수익 창출X\n출처 : 찰리푸스 틱톡,https://i.ytimg.com/vi/7t5JDCzWYOY/default.jpg,찰떡,"챨퓨, ckfvn, 찰푸, ckfflvntm, charlieputh",2d,hd,False,False,NaN,NaN
4,카타르월드컵 가나전 벤투 퇴장에 대한 진실 #shorts,-yque85ab-k,2023-09-21,17,1264017,26279,410,0,0,59,...,NaN,https://i.ytimg.com/vi/-yque85ab-k/default.jpg,리춘수 [이천수],NaN,2d,hd,False,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32731,저 회장직 내려놓겠습니다,PUzfkPdTqW0,2023-03-21,20,462409,2706,517,0,36,15,...,[아프리카TV] http://www.afreecatv.com/rrvv17\n[광고...,https://i.ytimg.com/vi/PUzfkPdTqW0/default.jpg,킴성태TV,"오버워치, 옵치, 서든, 서든어택, 킴성태TV, 배틀그라운드, 배그, 킴해적단, 박...",2d,hd,false,True,,
32732,밥에 비비면 맵싹달싹! 지코바!,vgxc9ca9Mwg,2023-04-19,22,445476,5210,648,0,7,25,...,#먹방 #쿡방 #지코바\n\n밥에 비비면 맵싹달싹! 지코바!,https://i.ytimg.com/vi/vgxc9ca9Mwg/default.jpg,핫식당,"핫식당, 먹방, 쿡방, 핫소스, 배말랭, 김선응, 송형주, 배건우, 밥에 비비면 맵...",2d,hd,false,True,,
32733,“저출산 문제에 진심”..무릎까지 꿇었다!,a8w5_ZvSTdo,2023-04-19,25,3907877,40503,8139,0,3,3,...,최정우 포스코그룹 회장이 국내 최초로 네쌍둥이 자연 분만에 성공한 직원의 집을 찾았...,https://i.ytimg.com/vi/a8w5_ZvSTdo/default.jpg,엠빅뉴스,"다둥이, 포스코, 출산, 분만, 산부인과, 카니발, 저출산, 출산장려금, 회장님, ...",2d,hd,false,True,,
32734,"[EN] 쩡수 하우스에서 간단하게 삼겹살 5kg 공깃밥 대신 꼬마김밥 Today, ...",PibUEH7Ing0,2021-11-19,22,2018580,26717,1725,0,12,16,...,광고 및 협업 문의 메일 : fucompany16@gmail.com\n\n오늘 맛본...,https://i.ytimg.com/vi/PibUEH7Ing0/default.jpg,윤코치의 맛만 볼까? Coach Yoon,"맛만볼까, 영암민속씨름단, 윤정수, 윤정수코치, 먹방, 삼겹살, 대패삼겹살, 꼬마김...",2d,hd,true,True,,
